In [ ]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce

## Get the latest email opt in/out data from DB2


In [3]:
# get new preferences for users
pcomms = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles 
where profile_key in ('personalizedcommunication');",nanohub_db)#order by rand() limit 50000;",engine) #and modifiedDate >= '2019-08-08'

up_news = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles \
where profile_key in ('updates_news');",nanohub_db) #order by rand() limit 50000;",engine)

KeyboardInterrupt: 

In [ ]:
display(pcomms.head(2))
display(pcomms.tail(2))
display(up_news.head(2))
display(up_news.tail(2))

In [ ]:
print(pcomms.shape)
print(up_news.shape)

In [ ]:
## adjusting new_email dataframe for the new email optin values 
# 0 or 1
pcomms_t = pcomms[['user_id','profile_value']]
up_news_t = up_news[['user_id','profile_value']]

display(pcomms_t.head(2))
display(up_news_t.head(2))

In [ ]:
## filtering pcomms_t with 0 or 1
ptt = pcomms_t['profile_value'].to_list()

for index, val in enumerate(ptt):
    val = val.split(',')[0]
    if val == 'Yes':
        ptt[index] = 1 # 1 indicates opt-in
    else:
        ptt[index] = 0 # 1 indicates opt-out

pcomms_t.loc[:,'profile_value'] = ptt


In [ ]:
## filtering up news t with 0 or 1
untt = up_news_t['profile_value'].to_list()

for index, val in enumerate(untt):
    val = val.split(',')[0]
    if val == 'Yes':
        untt[index] = 1 # 1 indicates opt-in
    else:
        untt[index] = 0 # 1 indicates opt-out

up_news_t.loc[:,'profile_value'] = untt


In [ ]:
## display
display(pcomms_t.head(2))
display(pcomms_t.tail(2))
display(up_news_t.head(2))
display(up_news_t.tail(2))

In [ ]:
## create new dataframe
new_opts = pd.DataFrame()

new_opts['nanoHUB_user_ID__c'] = pcomms_t['user_id']
new_opts['personalizedcommunication__c'] = pcomms_t['profile_value']
new_opts['updatesnews__c'] = up_news_t['profile_value']


In [ ]:
display(new_opts.head(2))
print(type(new_opts))

## Send to SF

In [ ]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(new_opts)


In [ ]:
# check status
db_s.check_bulk_status()

In [ ]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())